<a href="https://colab.research.google.com/github/Sunnnyyy16/EmoSupportAI/blob/main/SupportlyChat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



#SupportlyChat


*   LLaMa3.2-1B 모델을 MentalWell 데이터셋으로 학습
*   사용자 입력 기반 맞춤형 응답 제공
*   Low-Rank Adaptation (LoRA)






In [ ]:
!pip install datasets jsonlines

In [ ]:
import huggingface_hub
huggingface_hub.login()

In [ ]:
import pandas as pd

import json
import jsonlines

from datasets import Dataset

In [ ]:
import csv
import json

def csv_to_jsonl(csv_file_path, jsonl_file_path):
    with open(csv_file_path, 'r', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)

        with open(jsonl_file_path, 'w', encoding='utf-8') as jsonl_file:
            for row in csv_reader:
                json_line = json.dumps(row)
                jsonl_file.write(json_line + '\n')

input_file = 'MentalWell.csv'
output_file = 'MentalWell.jsonl'

try:
    csv_to_jsonl(input_file, output_file)
    print(f"Successfully converted {input_file} to {output_file}")
except Exception as e:
    print(f"An error occurred: {str(e)}")

In [ ]:
datasetName = "MentalWell.csv"
jsonFileName = "MentalWell.jsonl"

In [ ]:
def csv_to_json(csv_file_path, json_file_path):
    df = pd.read_csv(csv_file_path)

    with open(json_file_path, 'w', encoding='utf-8') as json_file:
        for index, row in df.iterrows():
            data = {'inputs': row['questionText'], 'response': row['new_answerText']}
            json.dump(data, json_file, ensure_ascii=False)
            json_file.write('\n')

csv_file_path = datasetName
json_file_path = jsonFileName

csv_to_json(csv_file_path, json_file_path)

In [ ]:
!pip install accelerate peft bitsandbytes transformers trl

  Using cached datasets-3.1.0-py3-none-any.whl.metadata (20 kB)
Using cached datasets-3.1.0-py3-none-any.whl (480 kB)
  Attempting uninstall: datasets
    Found existing installation: datasets 2.16.1
    Uninstalling datasets-2.16.1:
      Successfully uninstalled datasets-2.16.1


In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

In [ ]:
base_model = "meta-llama/Llama-3.2-1B"

code_dataset = "dhlim55/mental_health_happydb"

new_model = "llama-3.2-1b-formentalhealth_happydb"

In [ ]:
dataset = load_dataset(code_dataset, split="train")

train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/274 [00:00<?, ?B/s]

In [ ]:
print( dataset[28] )

{'text': '<s>[INST] I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac.    I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years.\n   I’ve never had counseling about any of this. Do I have too many issues to address in counseling? [/INST] First, thank you for being open and vulnerable by sharing with us online. Second, many of these "issues" you have mentioned are all interconnected. Often my clients have found that addressing past traumas helps diminish their depression. Which in turn can diminish the anxiety. The first step to wellness is to take a chance on ourselves and seek out a clinician we can connect with. Namaste Enjoy a hands-on hobby, like flying an RC plane—finding fun in small challenges brings light-hearted joy. </s>'}


In [ ]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

In [ ]:
trainer.train()

Step,Training Loss
25,2.767800
50,2.740600
75,2.666300
100,2.629000
125,2.646800
150,2.639300
175,2.591000
200,2.606200
225,2.550400
250,2.632800


TrainOutput(global_step=3915, training_loss=2.2025169645384963, metrics={'train_runtime': 1076.4864, 'train_samples_per_second': 7.274, 'train_steps_per_second': 3.637, 'total_flos': 1.363036785205248e+16, 'train_loss': 2.2025169645384963, 'epoch': 3.0})

In [ ]:
import re

logging.set_verbosity(logging.CRITICAL)

prompt = "I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac. \
          I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years. \
          I’ve never had counseling about any of this. Do I have too many issues to address in counseling?"

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

result = pipe(
    f"<s>[INST] {prompt} [/INST]",
    pad_token_id=tokenizer.eos_token_id,
    no_repeat_ngram_size=2,
)

response = result[0]['generated_text']

# 마지막 생성되다가 만 문장 삭제
sentences = re.split(r'(?<=\.)\s+', response)

if not sentences[-1].endswith('.'):
    sentences.pop()

final_response = ' '.join(sentences)

print(final_response + ' Here is the website address for a prominent psychological counseling center: https://www.mykpcc.com/. \
You can visit this site or call 129 to receive counseling from a professional counselor.')

<s>[INST] I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac. I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years. I’ve never had counseling about any of this. Do I have too many issues to address in counseling? [/INST] No, you do not have "too many" issues. In fact, many people have many "issues" that need to be addressed. The key is to find a therapist who can help you to work through them. You do have enough to deal with to start counseling. Try a few minutes of meditation; it might helpyou find calm and inner peace. </s> Reconnect with someone you haven’t spoken to in a while—it can be a special and joyful experience for both of you. Here is the website address for a prominent psychological counseling center: https://www.mykpcc.com/. You can visit this site or call 129 to receive counseling from a professional counse

In [ ]:
#유저별 실험
import re

logging.set_verbosity(logging.CRITICAL)

prompt = "Hello, I’m seeking advice about a recent concern regarding a friendship.\
 My best friend has started dating someone, but I don’t have a good impression of her boyfriend.\
  He seems to have a violent temperament and often speaks harshly to her. \
  Last week, the three of us met up, and he even shouted at her in a public cafe.\
   I want to tell my friend that it would be better for her not to see him,\
    but I’m afraid this might strain our friendship. What is the best course of action I can take in this situation?"

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=300)

result = pipe(
    f"<s>[INST] {prompt} [/INST]",
    pad_token_id=tokenizer.eos_token_id,
    no_repeat_ngram_size=2,
)

response = result[0]['generated_text']

# 마지막에 생성되다가 만 문장 삭제하기
sentences = re.split(r'(?<=\.)\s+', response)

if not sentences[-1].endswith('.'):
    sentences.pop()

final_response = ' '.join(sentences)

print(final_response + ' Here is the website address for a prominent psychological counseling center: https://www.mykpcc.com/. \
You can visit this site or call 129 to receive counseling from a professional counselor.')